# PySPH Tutorial: without built-in schemes

*Prabhu Ramachandran*

-----

## Recap

- Looked at creating a simple problem
- Saw how to use schemes and solve the elliptical drop example.
- Looked at simple post-processing of the data.


Schemes provide convenient pre-built particle interactions.

So how do you do your own thing without schemes.

Let us now do it in a more low-level way.  To do this one needs the following methods:

- `create_particles(self)`
- `create_equations(self)`
- `create_solver(self)`

In [ ]:
import numpy as np

from pysph.solver.application import Application
from pysph.base.utils import get_particle_array_wcsph

class EllipticalDrop(Application):
    def create_particles(self):
        # ...
        pass
        
    def create_equations(self):
        pass
    
    def create_solver(self):
        pass

    
if __name__ == '__main__':
    app = EllipticalDrop()
    app.run()

### Exercise

- Create an `ed_no_scheme.py` which uses this skeleton, copy over your completed `create_particles` method
- Don't execute the example yet.

Now, let us flesh out the other methods.  Add this code:


In [ ]:
from pysph.sph.equation import Group
from pysph.sph.basic_equations import XSPHCorrection, ContinuityEquation
from pysph.sph.wc.basic import TaitEOS, MomentumEquation

class EllipticalDrop(Application):
    def create_equations(self):
        equations = [
            Group(
                equations=[
                    TaitEOS(dest='fluid', sources=None, rho0=1.0,
                            c0=1400, gamma=7.0),
                ],
                real=False
            ),

            Group(
                equations=[
                    ContinuityEquation(dest='fluid',  sources=['fluid']),

                    MomentumEquation(dest='fluid', sources=['fluid'],
                                     alpha=0.1, beta=0.0, c0=1400),

                    XSPHCorrection(dest='fluid', sources=['fluid']),
                ]
            ),
        ]
        return equations

- Note that the `Group` allows one to group a set of equations together.
- All equations in a group are completed first before going on to the next group.
- Explore the different equations.
- Note that we return a list of equations.


Next we create the solver:

In [ ]:
from pysph.base.kernels import CubicSpline

from pysph.solver.application import Application
from pysph.solver.solver import Solver
from pysph.sph.integrator import PECIntegrator
from pysph.sph.integrator_step import WCSPHStep

class EllipticalDrop(Application):
    def create_solver(self):
        kernel = CubicSpline(dim=2)

        # Note that fluid is the name of the fluid particle array.
        integrator = PECIntegrator(fluid=WCSPHStep())

        dt = 5e-6
        tf = 0.0076
        solver = Solver(
            kernel=kernel, dim=2, integrator=integrator,
            dt=dt, tf=tf
        )
        return solver

Let us understand the above a bit.

In [ ]:
from pysph.sph.integrator import EPECIntegrator

In [ ]:
EPECIntegrator??

#### Exercise

- Now put everything together and get it working.

#### Solution

If you are stuck, look at the solution below.

In [ ]:
%load solutions/ed_no_scheme.py

## Understanding equations

Let us understand equations a little more.

- Look at the `TaitEOS` class source
- Look at the `ContinuityEquation` class source.

- Understand the terms "source" and "destination"

Note the following:

- The methods `initialize`, `loop`, `post_loop` are called per-particle.
    - `initialize`: iterates over the destination particles
    - `loop`: iterates over the neighbors for each destination.
    - `post_loop`: iterates over the destination.
    
- `d_*` refers to destination props
- `s_*` refers to source props
- `d_idx`, `s_idx`, refer to particle indices.


## Understanding integrators and steppers


- Look at `PECIntegrator`
- Look at `WCSPHStep`
- Look at the `Solver` options.


- Some useful solver options:

    - kernel
    - integrator
    - dt, tf
    - adaptive_timestep: bool
    - cfl: float
    - n_damp: int
    - output_at_times: list

- API docs: http://pysph.readthedocs.io/en/latest/reference/

### Doing more

Many other options in the `Application` class.

- `add_user_options`: this is used to create additional user-defined command line arguments. The command line options are available in self.options and can be used in the other methods.
- `consume_user_options`: this is called after the command line arguments are parsed, and can be optionally used to setup any variables that have been added by the user in add_user_options. Note that the method is called before the particles and solver etc. are created.
- `create_domain`: this is used when a periodic domain is needed.
- `create_inlet_outlet`: Override this to return any inlet an outlet objects. See the pysph.sph.simple_inlet_outlet module.

See here:

http://pysph.readthedocs.io/en/latest/reference/application.html#pysph.solver.application.Application
